In [1]:
import os

In [2]:
%pwd

'd:\\SL\\End to End Projects\\New folder\\research'

In [3]:
import os

os.chdir("../")

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataValidationConfig:
    root_dirs: Path
    STATUS_FILE: str
    unzip_data_dir: Path
    all_schema: dict

In [5]:
from mlproject.constants import *
from mlproject.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH
    ):
         self.config = read_yaml(config_filepath)
         self.params = read_yaml(params_filepath)
         self.schema = read_yaml(schema_filepath)
         print(self.schema)
         create_directories([self.config.artifacts_root])
    
    def data_validation_config(self):
        config = self.config.data_validation
        schema = self.schema.COLUMNS
        
        create_directories([config.root_dirs])
        
        create_validation_config = DataValidationConfig(
            root_dirs= config.root_dirs,
            STATUS_FILE=config.STATUS_FILE,
            unzip_data_dir=config.unzip_data_dir,
            all_schema=schema
        )
        
        return create_validation_config
        


In [7]:
import pandas as pd

class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config
        
    
    def validate_all_columns(self) -> bool:
        try:
            validation_status = None
            data = pd.read_csv(self.config.unzip_data_dir)
            all_columms =list(data.columns)
            
            all_schema = self.config.all_schema.keys()

            for col in all_columms:
                if col in all_schema:
                    if data[col].dtype == self.config.all_schema[col]:
                        validation_status = True
                    else:
                        validation_status = False
                        
                    with open(self.config.STATUS_FILE,"w") as f:
                        f.write(f"validation status: {validation_status}")
                else:
                    
                    validation_status =False
                    with open(self.config.STATUS_FILE,"w") as f:
                        f.write(f"validation status: {validation_status}")
            return validation_status
        
        except Exception as e:
            raise e

In [8]:
try:
    config = ConfigurationManager()
    data_validation_config = config.data_validation_config()
    data_validation = DataValidation(config=data_validation_config)
    validate=data_validation.validate_all_columns()
    print(validate)
except Exception as e:
    raise e

[2024-07-29 12:35:32,006]: yaml file: config\config.yaml loaded successfully:
[2024-07-29 12:35:32,008]: yaml file: params.yaml loaded successfully:
[2024-07-29 12:35:32,012]: yaml file: schema.yaml loaded successfully:
{'COLUMNS': {'Date': 'object', 'Production': 'int64'}, 'TARGET_COLUMN': {'Production': 'int64'}}
Directory 'artifacts' created successfully or already exists.
[2024-07-29 12:35:32,013]: Created directory at: artifacts:
Directory 'artifacts/data_validation' created successfully or already exists.
[2024-07-29 12:35:32,014]: Created directory at: artifacts/data_validation:
dict_values(['object', 'int64'])
schema:  dict_keys(['Date', 'Production'])
columns:  ['Date', 'Production']
object == object
int64 == int64
True
